In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
# from itertools import product
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = os.path.join('/content/gdrive/MyDrive/Hackathon')
GOOGLE_DRIVE_PATH = os.path.join('gdrive', 'MyDrive',GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print('List files: ', os.listdir(GOOGLE_DRIVE_PATH))
data_file = os.path.join(GOOGLE_DRIVE_PATH,'clinical_dataset.csv')

List files:  ['clinical_dataset.csv', 'Harmony_hack.ipynb']


In [6]:
df = pd.read_csv(data_file)

In [7]:
df.columns

Index(['Unnamed: 0', 'id', 'nct_id', 'outcome_type', 'measure', 'time_frame',
       'population', 'description'],
      dtype='object')

In [8]:
df.head(2)

,Unnamed: 0,id,nct_id,outcome_type,measure,time_frame,population,description
0,0,272006148,NCT03556657,primary,Change from baseline in Adult Sickle Cell Qual...,Pre-test at baseline and post-test at 10 weeks,NaN,A patient-reported outcome measurement system ...
1,1,273155865,NCT01812382,secondary,Safety and tolerability of the microdialysis t...,Single day,NaN,"Laboratory parameters include: Hematology, Cli..."


In [9]:
df2 = df.copy()

df2.drop(columns = ['id','population'])

,Unnamed: 0,nct_id,outcome_type,measure,time_frame,description
0,0,NCT03556657,primary,Change from baseline in Adult Sickle Cell Qual...,Pre-test at baseline and post-test at 10 weeks,A patient-reported outcome measurement system ...
1,1,NCT01812382,secondary,Safety and tolerability of the microdialysis t...,Single day,"Laboratory parameters include: Hematology, Cli..."
2,2,NCT01751113,primary,Area Under the Curve Calculated From 0 to 4 Ho...,Day 28 of each treatment period (up to 35 days),sGaw is a measure of airways conductance and i...
3,3,NCT05834790,primary,Baseline oral healthcare-related knowledge,This outcome will be measured before intervent...,Fifteen statements will be used to measure par...
4,4,NCT01813474,secondary,Tmax Following Single Dosing,"Day 1: pre-dose, 0.25, 0.5, 1, 1.5, 2, 3, 4, 6...",NaN
...,...,...,...,...,...,...
9995,9995,NCT00579280,secondary,Change From Baseline in Hamilton Anxiety Scale...,8 weeks,Hamilton Anxiety Scale (HAM-A) measures severi...
9996,9996,NCT02111005,primary,P53 levels in gingival biopsy samples,The measurement will be performed at T0 (basel...,This variable is going to be measured by an im...
9997,9997,NCT05282628,primary,Child Behavior Checklist,Week 1,Child Behavior
9998,9998,NCT02193178,primary,Lens Fit - Overall Stability,"2 weeks prior to baseline, Baseline, 2 weeks p...",Lens Fit (stability) for habitual lenses asses...


In [10]:
df3 =df2.groupby('outcome_type')

In [11]:
# Separate DataFrames using boolean indexing
df3_primary = df[df['outcome_type'] == 'primary']
df3_secondary = df[df['outcome_type'] == 'secondary']
df3_other = df[df['outcome_type'] == 'other']

In [12]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import networkx as nx
import pandas as pd

# Load pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def encode_sentences(sentences, tokenizer, model):
    inputs = tokenizer(sentences, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

def compute_similarity_matrix(embeddings):
    similarity_matrix = cosine_similarity(embeddings)
    return similarity_matrix

def cluster_sentences(similarity_matrix, threshold=0.35):
    # Create a graph
    G = nx.Graph()

    # Add nodes
    num_sentences = similarity_matrix.shape[0]
    G.add_nodes_from(range(num_sentences))

    # Add edges based on the similarity threshold
    for i in range(num_sentences):
        for j in range(i+1, num_sentences):
            if similarity_matrix[i, j] > threshold:
                G.add_edge(i, j)

    # Find connected components (clusters)
    clusters = list(nx.connected_components(G))
    return clusters



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
# Data Cleaner Function
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Download NLTK stopwords if not already downloaded
import nltk
nltk.download('stopwords')
nltk.download('punkt')
custom_words =[
'sdata','newlinetoken','edata'
]
# Data preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercase
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]  # Lemmatization and remove non-alphabetic tokens
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

def remove_words(words, custom_words):
    # Initialize an empty list to store non-matching words
    remaining_words = []

    # Iterate through each word in the list
    for word in words:
        # Check if any of the substrings exist in the word
        if not any(custom_word in word for custom_word in custom_words):
            # If none of the substrings are present, add the word to the list of remaining words
            remaining_words.append(word)

    return remaining_words
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    filtered_words = remove_words(filtered_tokens,custom_words)
    # Join the filtered tokens back into a string
    cleaned_text = ' '.join(filtered_words)
    return cleaned_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer
df3_primary['measure_cleaned'] =df3_primary['measure'].apply(clean_text)

<ipython-input-15-712d5d67c5d6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_primary['measure_cleaned'] =df3_primary['measure'].apply(clean_text)


In [16]:
# Sample DataFrame
# df3_primary = pd.DataFrame(data)

# Encode sentences
sentences = df3_primary['measure_cleaned'].tolist()
embeddings = encode_sentences(sentences, tokenizer, model)

In [17]:
# Compute similarity matrix
similarity_matrix = compute_similarity_matrix(embeddings)
# Convert similarity matrix to DataFrame for better visualization
similarity_df = pd.DataFrame(similarity_matrix, columns=df3_primary.index, index=df3_primary.index)

# Display the similarity matrix
#print(similarity_df)

In [18]:
# Define thresholds
thresholds = np.arange(0.50, 1.05, 0.10)

# Find pairs with similarity above each threshold
similar_pairs_dict = {threshold: [] for threshold in thresholds}

# Use NumPy to find indices of pairs above each threshold
for threshold in thresholds:
    indices = np.where(similarity_matrix >= threshold)
    for i, j in zip(*indices):
        if i < j:  # to avoid duplicate pairs and self-comparison
            similar_pairs_dict[threshold].append((i, j, similarity_matrix[i, j]))



In [19]:
# Categorize similar sentences based on the highest threshold they meet
categories = {}
for threshold in thresholds:
    for i, j, similarity in similar_pairs_dict[threshold]:
        if i not in categories:
            categories[i] = []
        categories[i].append((j, threshold, similarity))

In [20]:
# Display categorized sentences
print("\nCategorized sentences based on the highest threshold they meet:")
for category, values in categories.items():
    print(f"Category for Sentence {category + 1}:")
    print(df['measure'][category])
    for value in values:
        index, threshold, similarity = value
        #print(f"  Similar Sentence {index + 1} with similarity {similarity:.2f} at threshold {threshold}")

Streaming output truncated to the last 5000 lines.
Category for Sentence 692:
Difference in biomarkers of cancer (change from baseline to 12-week follow-up)
Category for Sentence 693:
concentration of testosterone in plasma
Category for Sentence 694:
Mean LIC According to the Presence or Absence of Retrospective Hepatic Events
Category for Sentence 695:
Time from enrollment date to the onset of composite cerebrovascular events
Category for Sentence 696:
neuroendocrine (CRH) and immunological mediators (IL-1β, IL-6, Il-8, IL-10, IL-12, TNF-α, IFN-γ, G-CSF, GM-CSF)
Category for Sentence 697:
Adverse event(AE)
Category for Sentence 698:
Patients' satisfaction
Category for Sentence 699:
Superficial infection or Deep infection, defined as Synovial White Blood Cell (WBC) count > 4200 WBC/ml or Synovial WBC > 3000 WBC/ml & C-Reactive Protein (CRP) > 10 mg/dl & Erythrocyte Sedimentation Rate (ESR) > 30 mm/hr ;
Category for Sentence 700:
Change in diabetes support
Category for Sentence 701:
Bod

In [21]:
import pandas as pd

categorized_sentences = []

for category, values in categories.items():
    for value in values:
        index, threshold, similarity = value
        categorized_sentences.append({
              'Sentence_category': df['measure'][category],
              'Sentence_index': df['measure'][index],
              'Similarity': similarity,
              'Threshold': threshold
         })

categorized_df = pd.DataFrame(categorized_sentences)
categorized_df


,Sentence_category,Sentence_index,Similarity,Threshold
0,Change from baseline in Adult Sickle Cell Qual...,Safety and tolerability of the microdialysis t...,0.863903,0.5
1,Change from baseline in Adult Sickle Cell Qual...,Area Under the Curve Calculated From 0 to 4 Ho...,0.853759,0.5
2,Change from baseline in Adult Sickle Cell Qual...,Baseline oral healthcare-related knowledge,0.747487,0.5
3,Change from baseline in Adult Sickle Cell Qual...,Tmax Following Single Dosing,0.842666,0.5
4,Change from baseline in Adult Sickle Cell Qual...,Patient-centered Outcomes - Satisfaction,0.828900,0.5
...,...,...,...,...
15959530,Frequency of side effects of opioids,Diagnostic Yield of EUS-FNB and EUS-FNA,0.715669,0.7
15959531,Frequency of side effects of opioids,ARI with microbiologically confirmed influenza...,0.732320,0.7
15959532,Diagnostic Yield of EUS-FNB and EUS-FNA,ARI with microbiologically confirmed influenza...,0.729068,0.5
15959533,Diagnostic Yield of EUS-FNB and EUS-FNA,ARI with microbiologically confirmed influenza...,0.729068,0.6


In [22]:
categorized_df.sort_values("Similarity", inplace=True, ascending=False)
categorized_df

,Sentence_category,Sentence_index,Similarity,Threshold
14993750,number of correct femoral tunnel position,Relapse,1.000000,0.9
11966758,Skin melanin index,number of correct femoral tunnel position,1.000000,0.5
11970499,Skin melanin index,Relapse,1.000000,0.9
14992634,number of correct femoral tunnel position,Relapse,1.000000,0.6
11970501,Skin melanin index,number of correct femoral tunnel position,1.000000,1.0
...,...,...,...,...
7570656,Lower body muscle strength,Immune Overall Response Rate (iORR),0.500183,0.5
2440585,Cmax of free dabigatran,treatment response measured by ADAS-cog,0.500176,0.5
5636422,Incident prostate cancer,Myocardial infarction,0.500166,0.5
7570768,Lower body muscle strength,Age (years),0.500105,0.5


In [23]:
categorized_df.groupby("Sentence_category")[["Sentence_index", "Similarity"]].first().reset_index()

,Sentence_category,Sentence_index,Similarity
0,"""C-PROM"" questionnaire to measure Personal Rec...",Time to reach the MSSBP target (based on the B...,0.877558
1,% of patients with radiological improvement,Assessment of Area Under the Plasma Concentrat...,0.918555
2,% of patients with treatment failure defined a...,Survey,0.910103
3,(Phase 3) Change From Baseline in the Function...,Symptomatic Pulmonary Embolism (PE) at 36 Months,0.944611
4,- Heart rate and cardiac output,Nomadism indicator (potential risk factor),0.910598
...,...,...,...
3120,ventilatory stability by frequency of central ...,Progress free survival (PFS) per RECIST v1.1,0.921258
3121,worst grade toxicity per patient,Change in Functional Outcomes as Measured by P...,0.900503
3122,λz,Anxiety and Depression,0.901621
3123,• Fall in Faecal Calprotectin,Need for facilitated dental treatment and foll...,0.888675
